# Shor's Algorithm on Tket

<u><b>GOAL:</b></u> Find a number $p$ that divides $N$. <br>
<u><b>INPUT:</b></u> A positive integer $N$. <br>
<u><b>OUTPUT:</b></u> A positive integer $p$ that divides $N$.

## Classical Part

1. Pick a random number $a < N$ 
2. Find the greatest common divisor of $a$ and $N$.
> The Greatest Common Divisor (GCD) of two numbers is the largest positive integer that divides both of them. $GCD(a,b)$ can be found very fast with the Euclidean algorithm in $\mathcal{O}(log(min(a,b)))$.
3. If $GCD(a,N) \neq 1$ then we output $p=a$. Otherwise, run period finding subroutine to obtain a period $r$.
> $r$ is the smallest integer for which $f(x + r) = f(x)$, where $f(x) = a^x mod N$. The idea is that we want to find $r$ because: 

$$(a^r -1) mod N = 0.\$$

4. Then factors of $N$ should be $gcd(a^\frac{r}{2} ± 1, N)$.

> However, if $r$ is odd we need to go back to step 1 because since $a$ is also odd, then $ar$ will be odd and we will not get an integer when performing the division $\frac{ar}{2}$ on the last step.

> Also, since $(a^r-1) mod N = 0$, this imples that $a^r-1$ can be a any multiple of N, and therefore we also have to check that $a^{r/2} ≡ -1 (mod N)$. If this is the case then go back to step 1. 

## Quantum Part

> TODO: Talk about period finding subroutine

### Period finding

Traditionally, we would implement a function $f: \{0,1\}^n = [0, N-1] \longrightarrow \{0,1\}^n = [0, N-1]$ with an oracle gate $U_f$ defined as follows:
$$
f(x)=y \iff U_f \left|q \right>_{ancilla}\left|x\right>_{input} = \left|q \right>_{ancilla}\left|f(x) \oplus q \right>_{input}
$$
However, for the purposes of finding the period of a function, there is a more efficient way to encode it using only $n$ instead of $2n$ qubits:
$$
U_f\left|f(0) \right>=\left|f(1) \right> \\
U_f\left|f(1) \right>=\left|f(2) \right> \\
U_f\left|f(2) \right>=\left|f(3) \right> \\
\cdots \\
U_f\left|f(r-2) \right>=\left|f(r-1) \right> \\
U_f\left|f(r-1) \right>=\left|f(r)=f(0) \right>
$$
This defines $U_f$ for inputs $x \in f[0, N-1]$. We still have total freedom on how to define $U_f$ on the remaining inputs.

For our particular $f_a(x)=a^x \mod N$, we have:

$$
U_f\left|x\right>=\left|ax \mod N \right>
$$




[comment]: <> (Because a and N are coprime. Write it here if you havent't said it in the classical part)



However, if the function $f(x)=a^x \mod N$ is bijective from $\{0, N-1\}$ to $\{0, N-1\}$, the encoding can be performed in a simpler way using $n$ qubits instead of $n+n=2n$ qubits:
$$
U_f\left|x\right>=\left|f(x)\right>=\left|a^x \mod N\right>
$$

This gate $U_f$ has eigenvalues $e^{2\pi i \frac{s}{r}}$

In fact, 

By performing quantum phase estimation on it you can extract $r$

In [1]:
import math
import numpy as np
N=35
n=8
a=3

def f(x:int):
    return pow(a,x) % N
def is_injective(f, rng):
    values_of_f= set()
    for _input in rng:
        a=f(_input)
        values_of_f.add(a)
        
    return len(values_of_f) == len(rng)
xrange=range(64)
print(is_injective(f, xrange))

False


# Period finding (attempt 2)


Nice property of the function $f_a(x)$: it has no accidental degeneracies
Define a $n$-qubit quantum gate $U_f$ as follows:

For $x \in$ the *main cycle* $\{1, a, \cdots\}$, $U_f$ acts by cycling the $\left|x\right>$s:
$$
U_f\left|f(0) \right>=\left|f(1) \right> \\
U_f\left|f(1) \right>=\left|f(2) \right> \\
U_f\left|f(2) \right>=\left|f(3) \right> \\
\cdots \\
U_f\left|f(r-2) \right>=\left|f(r-1) \right> \\
U_f\left|f(r-1) \right>=\left|f(r)=f(0) \right>
$$

For $x \in$ all the rest, $ [0, 2^n-1]\setminus f[0,2^n-1]$, $U_f$ is defined arbitrarily.

This cyclic property of $U_f$ on $f[0, 2^n-1]$ ensures that $U_f$ has eigenvectors of the form:
$$
\left|u_0\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} 1 \left|a^k \mod N\right>\\
\left|u_1\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i \cdot 1 /r} \left|a^k \mod N\right>\\
\left|u_2\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i\cdot 2 /r} \left|a^k \mod N\right>\\
\cdots \\
\left|u_{r-1}\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i\cdot (r-1) /r} \left|a^k \mod N\right>\\
$$
Plus other eigenvectors, which depend on the arbitrarily completed definition of $U_f$, which we don't give a shit about.
The idea is to use the Quantum Phase Estimation subroutine, which we'll talk about shortly, on one of the states of $TOBEDEFINED$ to get a phase $\Phi=\frac{s}{r}$, with $s$ a random integer.

Turning this relationship around, we have:

$$
\left|a^0 \mod N\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} 1 \left|u_k\right>\\
\left|a^1 \mod N\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i \cdot 1 /r} \left|u_k\right>\\
\left|a^2 \mod N\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i \cdot 2 /r} \left|u_k\right>\\
\cdots \\
\left|a^{r-1} \mod N\right>= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{2 \pi i \cdot (r-1) /r} \left|u_k\right>\\
$$

# Quantum Phase Extimation

Given a unitary $U$, with eigenvectors $U \left| \psi_{\theta} \right>=e^{2 \pi \phi}\left| \psi_{\theta} \right>$, QPE is a quantum algorithm that, when fed $\left| \psi \right>$, outputs $\phi$ as a classical output and $\left| \psi \right>$ itself as a quantum output.

Here is the circuit:

In [2]:
#INSERT HERE QPE CIRCUIT

More specifically, it outputs (to be completed) with $\left< \psi_{\phi} | \psi \right>$ probability amplitude

The time cost of the QPE algorithm is $O()$, given by the Quantum Fourier Transform in the algorithm.

Let's just focus on the first of those states in (insert ref), and apply QPE on it

$$
\left| 1 \right >= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} 1 \left|u_k\right>
$$

$\left| 1 \right >$ is an equal weight superposition of the eigenvectors, so applying QPE on it yields $\phi=\frac{s}{r}$ for a random $s$ with equal probability

Doing it a few times yields a few values of $s$:

$$
\frac{s_1}{r}, \; \frac{s_2}{r}, \; \cdots, \; \frac{s_m}{r} 
$$

From which r can be determined using a continued fractions algorithm